In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow
import seaborn as sns
import cv2
from tqdm import tqdm

In [2]:
train = pd.read_csv('dataset/train.csv')
test = pd.read_csv('dataset/test.csv')

In [3]:
train.head()

,Image,target
0,96.jpg,manipuri
1,163.jpg,bharatanatyam
2,450.jpg,odissi
3,219.jpg,kathakali
4,455.jpg,odissi


In [4]:
train['target'].unique()

array(['manipuri', 'bharatanatyam', 'odissi', 'kathakali', 'kathak',
       'sattriya', 'kuchipudi', 'mohiniyattam'], dtype=object)

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
datagen = ImageDataGenerator(rescale=1./255)
train_generator=datagen.flow_from_dataframe(
    dataframe=train,
    directory=".\\dataset\\train",
    x_col = "Image",
    y_col = "target",
    class_mode = "categorical",
    target_size=(128,128),
    batch_size=32
)

# test_generator = datagen.flow_from_dataframe(
#     dataframe=test,
#     directory=".\\dataset\\test",
#     x_col = "Image",
#     target_size=(128,128),
#     class_mode=None,
#     batch_size=32
# )

Found 364 validated image filenames belonging to 8 classes.


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Dropout


model = Sequential()
model.add(Conv2D(filters=100,kernel_size=(3,3),input_shape=(128,128,3),activation='relu'))
model.add(MaxPool2D(pool_size=(3,3)))

model.add(Conv2D(filters=200,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(3,3)))

model.add(Conv2D(filters=300,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(3,3)))

model.add(Flatten())
model.add(Dropout(0.5))

model.add(Dense(64,activation='relu'))
model.add(Dense(8,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [12]:
model.fit(
    train_generator,
    epochs=20,
    # validation_data=test_generator
)

  ...
    to  
  ['...']
Train for 12 steps
Epoch 1/20
12/12 [==============================] - 14s 1s/step - loss: 2.1151 - accuracy: 0.1181
Epoch 2/20
12/12 [==============================] - 11s 922ms/step - loss: 2.0730 - accuracy: 0.1346
Epoch 3/20
12/12 [==============================] - 11s 947ms/step - loss: 2.0584 - accuracy: 0.1676
Epoch 4/20
12/12 [==============================] - 12s 1s/step - loss: 2.0104 - accuracy: 0.2033
Epoch 5/20
12/12 [==============================] - 12s 963ms/step - loss: 1.8885 - accuracy: 0.2747
Epoch 6/20
12/12 [==============================] - 12s 994ms/step - loss: 1.6778 - accuracy: 0.3709
Epoch 7/20
12/12 [==============================] - 13s 1s/step - loss: 1.6159 - accuracy: 0.3846
Epoch 8/20
12/12 [==============================] - 15s 1s/step - loss: 1.5607 - accuracy: 0.4313
Epoch 9/20
12/12 [==============================] - 14s 1s/step - loss: 1.3570 - accuracy: 0.5220
Epoch 10/20
12/12 [==============================] - 12s 1s/st

In [13]:
model.save('new_model.h5')

In [14]:
test_img = []
path = 'dataset/test/'
for i in tqdm(test['Image']):
    final_path = os.path.join(path,i)
    img = cv2.imread(final_path)
    img = cv2.resize(img,(128,128))
    test_img.append(img)

100%|██████████| 156/156 [00:01<00:00, 142.48it/s]


In [21]:
test_data = np.array(test_img)
test_data = np.reshape(test_data,(test_data.shape[0],128,128,3))
prediction  = model.predict_classes(test_data)

In [22]:
prediction

array([1, 6, 0, 6, 0, 6, 0, 2, 6, 6, 6, 0, 1, 6, 6, 0, 6, 7, 4, 0, 6, 6,
       6, 6, 4, 0, 0, 6, 7, 0, 6, 6, 0, 6, 6, 0, 4, 6, 0, 6, 1, 0, 6, 0,
       1, 0, 6, 5, 6, 0, 6, 1, 1, 6, 6, 6, 4, 6, 6, 1, 0, 1, 1, 6, 0, 6,
       6, 0, 6, 0, 6, 5, 0, 7, 6, 6, 0, 6, 1, 1, 6, 5, 7, 6, 6, 0, 5, 6,
       4, 6, 6, 1, 6, 6, 6, 1, 6, 0, 6, 1, 6, 6, 6, 6, 6, 0, 6, 6, 1, 6,
       1, 6, 6, 6, 1, 6, 6, 6, 5, 6, 6, 6, 6, 1, 6, 6, 5, 0, 6, 1, 0, 4,
       0, 0, 0, 6, 6, 4, 1, 6, 6, 6, 6, 1, 6, 7, 6, 1, 4, 0, 6, 4, 6, 6,
       6, 6], dtype=int64)

In [23]:
Class_map={'manipuri':0, 'bharatanatyam':1, 'odissi':2 ,'kathakali':3, 'kathak':4, 'sattriya':5,
 'kuchipudi':6, 'mohiniyattam':7}
inverse_map={0:'manipuri', 1:'bharatanatyam', 2:'odissi' ,3:'kathakali',4: 'kathak', 5:'sattriya',
 6:'kuchipudi', 7:'mohiniyattam'}
train['target']=train['target'].map(Class_map)

In [25]:
submission = pd.DataFrame({'Image': test.Image})
submission.to_csv('new_output.csv',index=False)
submission['target'] = prediction
submission['target'] = submission['target'].map(inverse_map)

In [26]:
submission.to_csv('new_output.csv',index=False)